# Auto Loader 

# Source directory
Sample dataset

Source: https://docs.databricks.com/en/query/formats/parquet.html

In [0]:
%scala 
 
case class MyCaseClass(key: String, group: String, value: Int, someints: Seq[Int], somemap: Map[String, Int])
val dataframe = sc.parallelize(Array(MyCaseClass("a", "vowels", 1, Array(1), Map("a" -> 1)),
  MyCaseClass("b", "consonants", 2, Array(2, 2), Map("b" -> 2)),
  MyCaseClass("c", "consonants", 3, Array(3, 3, 3), Map("c" -> 3)),
  MyCaseClass("d", "consonants", 4, Array(4, 4, 4, 4), Map("d" -> 4)),
  MyCaseClass("e", "vowels", 5, Array(5, 5, 5, 5, 5), Map("e" -> 5)))
).toDF()
// now write it to disk
dataframe.write.mode("overwrite").parquet("dbfs:/mnt/demo_parquet/testParquet")

defined class MyCaseClass
dataframe: org.apache.spark.sql.DataFrame = [key: string, group: string ... 3 more fields]

Check files in new folder 

In [0]:
%python 
data = spark.read.parquet("dbfs:/mnt/demo_parquet/testParquet")
 
display(data)

key,group,value,someints,somemap
b,consonants,2,"List(2, 2)",Map(b -> 2)
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4)
c,consonants,3,"List(3, 3, 3)",Map(c -> 3)
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5)
a,vowels,1,List(1),Map(a -> 1)


## Auto Loader code

In [0]:
path = 'dbfs:/mnt/demo_parquet/testParquet'

(spark.readStream
        .format('cloudFiles')
        .option('cloudFiles.format', 'parquet')
        .option('cloudFiles.schemaLocation', 'dbfs:/mnt/demo/parquet_checkpoint')
        .load(path)
       .writeStream
        .option('checkpointLocation', 'dbfs:/mnt/demo/parquet_checkpoint')
        .table('testParquetUpdate')
)

Out[53]: <pyspark.sql.streaming.query.StreamingQuery at 0x7feeccfddf40>

In [0]:
%sql
SELECT * FROM testParquetUpdate;

key,group,value,someints,somemap,_rescued_data
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4),null
b,consonants,2,"List(2, 2)",Map(b -> 2),null
c,consonants,3,"List(3, 3, 3)",Map(c -> 3),null
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5),null
a,vowels,1,List(1),Map(a -> 1),null


Auto loader 

In [0]:
%sql
SELECT count(*) FROM testParquetUpdate

count(1)
5


Loading new parquet files\
1 records will be added

In [0]:
%scala
case class MyCaseClass(key: String, group: String, value: Int, someints: Seq[Int], somemap: Map[String, Int])
val dataframe = sc.parallelize(Array(MyCaseClass("z", "consonants", 5, Array(1, 2, 3), Map("z" -> 6)))).toDF()
// now write it to disk
dataframe.write.mode("overwrite").parquet("dbfs:/mnt/demo_parquet/testParquet")

defined class MyCaseClass
dataframe: org.apache.spark.sql.DataFrame = [key: string, group: string ... 3 more fields]

In [0]:
%sql
SELECT count(*) as rows_count FROM testParquetUpdate

rows_count
6


Auto loader works as expectes, we have 1 more records in table 'testParquetUpdate'